In [ ]:
#https://bbs.archlinux.org/viewtopic.php?id=243409
#https://gist.github.com/satreix/c01fd1cb5168e539404b
customrepo="/home/$USER/customrepo"
custompkgs="/home/$USER/custompkgs"
archlive="/home/$USER/archlive"
##Install archiso
sudo pacman -S archiso
#Copy archiso profile
mkdir -p $customrepo
mkdir -p $custompkgs
sudo cp -r /usr/share/archiso/configs/releng/ $archlive


#linux-firmware
cd $custompkgs
git clone https://aur.archlinux.org/linux-firmware-git.git
#Build the package
cd linux-firmware-git
makepkg -f
repo-add $customrepo/customrepo.db.tar.gz $custompkgs/linux-firmware-git/linux-firmware-git*.pkg.tar.zst

#yay
cd $custompkgs
sudo rm -rf yay
git clone https://aur.archlinux.org/yay.git
#Build the package
cd yay
makepkg -f
repo-add $customrepo/customrepo.db.tar.gz $custompkgs/yay/yay*.pkg.tar.zst
cp $custompkgs/yay/yay*.pkg.tar.zst $customrepo



#xmrig
#git clone https://github.com/xmrig/xmrig.git
git clone https://aur.archlinux.org/xmrig.git

#gnome
#gdm
#rtl8188eu
#rtl8812bu
#aircrack-ng
#hashcat
#hcxtools
#hcxdumptool
#fsearch

echo "
#linux-firmware-git
yay
" | sudo tee -a "$archlive/packages.x86_64"


#Yeah, this was the big step that I was missing. I needed to add both "8192cu-dkms" and "linux-headers" to "packages.x86_64". I also had to move #"8192cu-dkms-v4.0.2_9000-14-any.pkg.tar.xz" into "~/customrepo".

#This gave me a successful build of the ISO with the new driver. My USB WiFi adapter is continuing to give me trouble, even with the new driver, so I'm going to go with the #smartphone USB tethering option, as I mentioned above, because it seems to be the most beginner-friendly.

#Thanks for all the help everyone.


#You will want to be able to boot into X so you can have a pretty GUI and all, for this you will need to list the relevant Xorg packages (xorg-server, xorg-xinit, xorg-server-utils); a desktop environment, or window manager; you may also want to include all the xf86-video drivers so you can boot the USB on any computer regardless of what video chipset is in use. Don’t forgot about sound (alsa-utils), networking (wicd), fonts, a file manager, a terminal emulator, and sudo.
#To give you some ideas, you can get a full list of what you currently have installed on your host machine, by executing the following and picking out what you need.
#pacman -Qqe

#Edit pacman.conf
#sudo nano $archlive/pacman.conf
#I add this to pacman.conf:
echo "[customrepo]
SigLevel = Optional TrustAll
Server = file:///home/$USER/customrepo" | sudo tee -a "$archlive/pacman.conf" 

#Build the ISO
mkdir -p $archlive/airootfs/usr/share/fonts
cp -r /usr/share/fonts/* $archlive/airootfs/usr/share/fonts/
sudo rm -rf $customrepo/customrepo.db

customrepo="/home/$USER/customrepo"
custompkgs="/home/$USER/custompkgs"
archlive="/home/$USER/archlive"
cd $archlive
sudo mkdir out
sudo cp /usr/share/archiso/configs/releng/build.sh $archlive
sudo ./build.sh -v
#dd if=~/archlive/out/*.iso of=/dev/sdX




In [ ]:
#https://wiki.archlinux.org/index.php/Remastering_the_Install_ISO

set -k
yay -S --noconfirm arch-install-scripts squashfs-tools
isofn="manjaro-gnome-20.0.3-200606-linux56.iso"
mntdir="/mnt/archiso"
isodir="/run/media/$USER/SED/VBOX/ISOS"
isodir="/home/$USER/Documents"
isodir="/mnt/SED/VBOX/ISOS"
isofp="$isodir/$isofn"
customdir="/home/$USER/customiso"
dfs="desktopfs.sfs"

sudo mkdir -p $mntdir
sudo mkdir -p $customdir
#cp $isofp /home/$USER/Documents
sudo mount -t iso9660 -o loop $isofp $mntdir
cp -a $mntdir $customdir
#cd $customdir/arch/x86_64
cd $customdir/manjaro/x86_64
sudo rm -rf squashfs-root
sudo unsquashfs $dfs
sudo cp $customdir/boot/vmlinuz-x86_64 $customdir/manjaro/x86_64/squashfs-root/boot/vmlinuz-linux
sudo mkdir -p squashfs-root/{etc,proc,sys,dev/pts}
#sudo umount -l /proc
sudo mount --bind squashfs-root squashfs-root
sudo mount -t proc proc $customdir/manjaro/x86_64/squashfs-root/proc
#sudo mount -t proc none squashfs-root/proc
sudo mount -t sysfs none $customdir/manjaro/x86_64/squashfs-root/sys
sudo mount -o bind /dev squashfs-root/dev
sudo mount -o bind /dev/pts squashfs-root/dev/pts  ## important for pacman (for signature check)
sudo cp -L /etc/resolv.conf squashfs-root/etc  ## this is needed to use networking within the chroot
sudo arch-chroot squashfs-root /bin/bash

#https://forum.manjaro.org/t/virtualbox-wont-work-no-sbin-vboxconfig-file/96517/4
#chroot
pacman-key --init
pacman-key --populate archlinux
pacman -Syu --force archiso linux
nano /etc/mkinitcpio.conf
HOOKS="base udev memdisk archiso_shutdown archiso archiso_loop_mnt archiso_pxe_common archiso_pxe_nbd archiso_pxe_http archiso_pxe_nfs archiso_kms block filesystems keyboard"
mkinitcpio -p linux

#clear cache
LANG=C pacman -Sl | awk '/\[installed\]$/ {print $1 "/" $2 "-" $3}' > /pkglist.txt
pacman -Scc
exit

#if updating kernel
mv squashfs-root/boot/vmlinuz-linux $customdir/arch/boot/x86_64/vmlinuz
mv squashfs-root/boot/initramfs-linux.img $customdir/arch/boot/x86_64/archiso.img
rm squashfs-root/boot/initramfs-linux-fallback.img

mv squashfs-root/pkglist.txt ~/customiso/arch/pkglist.x86_64.txt

rm $dfs
mksquashfs squashfs-root $dfs

rm -r squashfs-root
sha512sum $dfs > airootfs.sha512

mkdir mnt
mount -t vfat -o loop ~/customiso/EFI/archiso/efiboot.img mnt
cp $customdir/arch/boot/x86_64/vmlinuz mnt/EFI/archiso/vmlinuz.efi
cp $customdir/arch/boot/x86_64/archiso.img mnt/EFI/archiso/archiso.img

dd if=/dev/zero bs=1M count=50 of=efiboot-new.img
mkfs.fat -n "ARCHISO_EFI" efiboot-new.img
mkdir new
mount -t fat -o loop efiboot-new.img new
cp -r mnt/* new/
umount new mnt
mv efiboot-new.img $customdir/EFI/archiso/efiboot.img

#ISO
genisoimage -l -r -J -V "ARCH_201209" -b isolinux/isolinux.bin -no-emul-boot -boot-load-size 4 -boot-info-table -c isolinux/boot.cat -o ../arch-custom.iso ./